# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from PositionalModel import PositionalModel

from src.match_names import *
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(FEATURES_OUTFIELD)

38

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape

(35410, 273)

In [7]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xG_30', 'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30',
       'Min_points_30', 'Team_result_30', 'DEF', 'FWD', 'MID', 'Finished'],
      dtype='object', length=273)

In [8]:
# df = df[df["GW"] == 2]

In [9]:
# X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]
df = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [10]:
df.loc[df["Finished"] == False, "Price"] = df["Price"] / 10

In [11]:
df.isna().sum().sort_values(ascending=False).head(20)

FPL_GW           16322
Min_points_2     16322
Min_points_4     16322
Min_points_30    16322
Min_points       16322
Clean_name       16322
xP               15808
Cmp%             15790
GCA              15786
npxG             15786
xA               15786
SCA              15786
PrgP             15786
Cmp              15786
Att              15786
Carries          15786
T_att            15786
T_succ           15786
xG               15786
PrgC             15786
dtype: int64

In [12]:
# df[FEATURES_OUTFIELD].iloc[0]

In [13]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [14]:
df.shape

(16262, 273)

In [15]:
X_test = df.copy()

In [16]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xG_30', 'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30',
       'Min_points_30', 'Team_result_30', 'DEF', 'FWD', 'MID', 'Finished'],
      dtype='object', length=273)

In [17]:
file = open("models/GBR_positional.pkl",'rb')
model = pickle.load(file)

In [18]:
model

In [19]:
model.predict( X_test.head(5) )

[2.498921957882968,
 2.0176981131235756,
 1.6432368199030558,
 3.0168577926223348,
 3.97400181461025]

In [20]:
model.predict( X_test.head(3) )

[2.498921957882968, 2.0176981131235756, 1.6432368199030558]

In [21]:
model.predict( X_test.head(4) )

[2.498921957882968, 2.0176981131235756, 1.6432368199030558, 3.0168577926223348]

In [22]:
X_test.head(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xG_30,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.03,1.02,1777.369124,1761.774146,NaN,1.166667,1.0,0.0,0.0,True
105,105,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,0.03,1.02,1773.661765,1765.611943,NaN,1.428571,1.0,0.0,0.0,True
106,106,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,0.03,1.02,1775.117339,1767.636719,NaN,1.413793,1.0,0.0,0.0,True
181,181,2023-08-12,Sat,Home,Arsenal,Nott'ham Forest,Aaron-Ramsdale,1.0,GK,90.0,...,0.00,1.18,1780.652535,1925.740381,NaN,2.100000,0.0,0.0,0.0,True
182,182,2023-08-21,Mon,Away,Arsenal,Crystal Palace,Aaron-Ramsdale,1.0,GK,90.0,...,0.00,1.18,1770.298556,1926.720374,NaN,2.100000,0.0,0.0,0.0,True


In [23]:
X_test.head(3)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xG_30,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.03,1.02,1777.369124,1761.774146,NaN,1.166667,1.0,0.0,0.0,True
105,105,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,0.03,1.02,1773.661765,1765.611943,NaN,1.428571,1.0,0.0,0.0,True
106,106,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,0.03,1.02,1775.117339,1767.636719,NaN,1.413793,1.0,0.0,0.0,True


# Predicting

In [25]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [26]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [27]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [28]:
X_test[FEATURES_OUTFIELD].isna()

,Was_home,Rating_difference,Price,Transfers_balance,Avg_FPL_points,Min_2,Gls_2,Ast_2,xG_2,xA_2,...,xG_30,xA_30,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,FPL_points_30,DEF,FWD,MID
76,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
105,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
181,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
182,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35405,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35406,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35407,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35408,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
# X_test.shape,  model.predict(X_test).shape

In [30]:
%%time
preds = get_predictions(model, df, X_test)

16262 (16262, 273)
CPU times: total: 2.67 s
Wall time: 1.36 s


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [31]:
# next gameweek
preds[preds["GW"] == 3].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5069,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2083.552734,1631.614380,FWD,14.0,6.021398
11634,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1946.371704,1884.362427,MID,12.5,5.158288
1513,André Onana,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1865.608521,1677.728027,GK,5.0,5.130172
12820,Pervis Estupiñán,3.0,2023-24,Brighton,West Ham,1.0,1843.608276,1787.674805,DEF,5.2,5.070448
5894,Guglielmo Vicario,3.0,2023-24,Tottenham,Bournemouth,0.0,1829.086060,1660.140991,GK,5.0,4.790428
10810,Martin Ødegaard,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,MID,8.5,4.786445
2554,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1865.608521,1677.728027,MID,8.5,4.702506
14466,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1843.608276,1787.674805,MID,6.6,4.692200
9267,Kieran Trippier,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1884.362427,1946.371704,DEF,6.5,4.561622
2088,Ben Mee,3.0,2023-24,Brentford,Crystal Palace,1.0,1835.432983,1758.575684,DEF,4.9,4.472690


In [32]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 3)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
1513,André Onana,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1865.608521,1677.728027,GK,5.0,5.130172
5894,Guglielmo Vicario,3.0,2023-24,Tottenham,Bournemouth,0.0,1829.086060,1660.140991,GK,5.0,4.790428
685,Aaron Ramsdale,3.0,2023-24,Arsenal,Fulham,1.0,1926.882202,1732.479858,GK,5.0,4.335727
4638,Ederson Santana de Moraes,3.0,2023-24,Manchester City,Sheffield United,0.0,2083.552734,1631.614380,GK,5.5,4.132880
8446,José Malheiro de Sá,3.0,2023-24,Wolves,Everton,0.0,1704.387207,1691.314331,GK,5.0,4.105067
4925,Emiliano Martínez Romero,3.0,2023-24,Aston Villa,Burnley,0.0,1825.121338,1722.802979,GK,5.0,4.053148
1153,Alisson Ramses Becker,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1946.371704,1884.362427,GK,5.5,3.755285
8267,Jordan Pickford,3.0,2023-24,Everton,Wolves,1.0,1691.314331,1704.387207,GK,4.5,3.586177
12280,Norberto Murara Neto,3.0,2023-24,Bournemouth,Tottenham,1.0,1660.140991,1829.086060,GK,4.5,3.446052
12172,Nick Pope,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1884.362427,1946.371704,GK,5.5,3.359244


In [33]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
5098,Erling Haaland,33.0,2023-24,Manchester City,Luton,1.0,2083.552734,1602.327515,FWD,14.0,6.543320
5070,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.552734,1732.479858,FWD,14.0,6.543320
5072,Erling Haaland,6.0,2023-24,Manchester City,Nott'ham Forest,1.0,2083.552734,1677.728027,FWD,14.0,6.543320
5095,Erling Haaland,30.0,2023-24,Manchester City,Arsenal,1.0,2083.552734,1926.882202,FWD,14.0,6.543320
5080,Erling Haaland,14.0,2023-24,Manchester City,Tottenham,1.0,2083.552734,1829.086060,FWD,14.0,6.543320
5092,Erling Haaland,27.0,2023-24,Manchester City,Manchester Utd,1.0,2083.552734,1865.608521,FWD,14.0,6.543320
5096,Erling Haaland,31.0,2023-24,Manchester City,Aston Villa,1.0,2083.552734,1825.121338,FWD,14.0,6.543320
5090,Erling Haaland,25.0,2023-24,Manchester City,Chelsea,1.0,2083.552734,1780.894531,FWD,14.0,6.543320
5089,Erling Haaland,24.0,2023-24,Manchester City,Everton,1.0,2083.552734,1691.314331,FWD,14.0,6.543320
5087,Erling Haaland,22.0,2023-24,Manchester City,Burnley,1.0,2083.552734,1722.802979,FWD,14.0,6.543320


In [34]:
preds[preds["FPL_pos"] == "GK"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
30,André Onana,1.0,2023-24,Manchester Utd,Wolves,1.0,1873.631104,1719.037964,GK,5.0,6.180947
1524,André Onana,14.0,2023-24,Manchester Utd,Newcastle Utd,0.0,1865.608521,1884.362427,GK,5.0,5.788769
1527,André Onana,17.0,2023-24,Manchester Utd,Liverpool,0.0,1865.608521,1946.371704,GK,5.0,5.788769
1537,André Onana,27.0,2023-24,Manchester Utd,Manchester City,0.0,1865.608521,2083.552734,GK,5.0,5.788769
1514,André Onana,4.0,2023-24,Manchester Utd,Arsenal,0.0,1865.608521,1926.882202,GK,5.0,5.788769
31,André Onana,2.0,2023-24,Manchester Utd,Tottenham,0.0,1877.037842,1817.656738,GK,5.0,5.739285
1547,André Onana,37.0,2023-24,Manchester Utd,Arsenal,1.0,1865.608521,1926.882202,GK,5.0,5.458861
1544,André Onana,34.0,2023-24,Manchester Utd,Newcastle Utd,1.0,1865.608521,1884.362427,GK,5.0,5.458861
1542,André Onana,32.0,2023-24,Manchester Utd,Liverpool,1.0,1865.608521,1946.371704,GK,5.0,5.458861
1520,André Onana,10.0,2023-24,Manchester Utd,Manchester City,1.0,1865.608521,2083.552734,GK,5.0,5.458861


In [35]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Erling Haaland,230.200861
Mohamed Salah,210.128053
André Onana,201.133138
Kieran Trippier,193.685399
Guglielmo Vicario,184.773158
Trent Alexander-Arnold,175.658129
Solly March,167.580441
Andrew Robertson,162.283220
Martin Ødegaard,161.950092


# Saving to file

In [36]:
preds.to_csv("predictions/preds_next_season.csv", index=False)